### **Generación de Texto con una RNN** ###

Vamos a utilizar una RNN basada en caracteres para [generar texto](https://www.tensorflow.org/text/tutorials/text_generation). Le mostraremos a la red una muestra de lo que queremos y aprenderá cómo escribir una versión por si misma. Para hacerlo, vamos a utilizar un modelo de predicción de caracteres que tomará como entrada una secuencia de longitud variable y predice el siguiente caráter. Si lo utilizamos de manera recurrente, conseguiremos que se cree un texto.

**Modulos:**

In [3]:
import os
import numpy as np
import time
import tensorflow as tf

**Dataset:**

Vamos a utilizar como Dataset de entrada para entrenar nuestro modelo parte de la obra Romeo y Julieta de Shakespeare.

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

**Lectura de Datos:**

In [ ]:
# Cargamos los datos de entrada a partir del fichero
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Logitud del texto: {len(text)} caracteres')
print(text[:250])

In [10]:
vocab = sorted(set(text)) # Ordenamos cada carácter único existente en el texto e entrada
print(f'{len(vocab)} caracteres únicos')

65 caracteres únicos


**Preprocesar la entrada de datos:**

La entrada de datos está en formato texto. Antes del entrenamiento, hay que convertir las cadenas en una representación numérica.

La capa tf.keras.layers.StringLookup puede convertir cada carácter en un ID numérico. Solo es necesario que el texto se divida en tokens primero. Para ello podemos utilizar la función tf.strings.unicode_split... Ejemplo...

In [13]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars # La b por delante de cada carácter representa que son cadenas de bytes

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Creamos la capa [tf.keras.layers.StringLookup](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup)

In [19]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [20]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Dado que nuestro objetivo es generar texto, es necesario invertir esta representación y recuperar cadenas legibles por humanos a partir de ella. Para esto, se puede usar [tf.keras.layers.StringLookup](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup)(..., invert=True).

Aquí, en lugar de pasar el vocabulario original generado con sorted(set(text)), usamos el método get_vocabulary() de la capa tf.keras.layers.StringLookup para que los tokens [UNK] se configuren de la misma manera.

In [21]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Esta capa recupera los caracteres de los vectores de ID y los devuelve como un tf.RaggedTensor de caracteres.

In [22]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Podemos usar tf.strings.reduce_join para volver a unir los caracteres en cadenas.

In [23]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [24]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

**La tarea de predicción:**

Dado un carácter, o una secuencia de caracteres, ¿cuál es el próximo carácter más probable? Ésta es la tarea para la que estamos entrenando al modelo. La entrada al modelo será una secuencia de caracteres, y entrenamos el modelo para predecir la salida: el siguiente carácter en cada paso de tiempo.

Dado que las RNN mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres computados hasta este momento, ¿cuál es el siguiente carácter?

**Crear ejemplos y objetivos de entrenamiento:**

Dividimos el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá caracteres *seq_length* del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Dividimos el texto en partes de *seq_length*+1 . Por ejemplo, digamos que seq_length es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hell" y la secuencia de destino "ello".

Para hacer esto, primero usamos la función tf.data.Dataset.from_tensor_slices para convertir el vector de texto en una secuencia de índices de caracteres.

In [25]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [28]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

El método batch nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.

In [29]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


Podemos volver a unir los tokens en cadenas para que sea más sencillo visualizar lo que estamos haciendo.

In [30]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para el entrenamiento, necesitamos un conjunto de datos de (input, label). Donde input y label son secuencias. En cada paso de tiempo, la entrada es el carácter actual y la etiqueta es el siguiente carácter.

Creamos una función que toma una secuencia como entrada, la duplica y la cambia para alinear la entrada y la etiqueta para cada paso de tiempo:

In [31]:
def split_input_target(sequence):   # Ejemplo: hello
    input_text = sequence[:-1]      # hell
    target_text = sequence[1:]      # ello
    return input_text, target_text  # hell, ello

In [32]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [33]:
dataset = sequences.map(split_input_target)

In [34]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Crear lotes de entrenamiento:**

Una vez dividido el texto en secuencias manejables, hay que mezclar los datos y empaquetarlos en lotes antes de introducirlos en el modelo.

In [35]:
BATCH_SIZE = 64     # Cantidad de ejemplos para training por Batch

BUFFER_SIZE = 10000 # Cantidad de elementos de la secuencia que considera para el Suffle

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**Construcción del Modelo:**

Definimos el modelo como una subclase keras.Model (para más info [Creación de nuevas capas y modelos a través de subclases](https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing) ).

Creamos un modelo con 3 capas: La capa de embedding se encarga de la representación de las palabras, la capa GRU captura patrones secuenciales, y la capa densa genera la salida final del modelo.

* tf.keras.layers.Embedding: La capa de entrada. Una tabla de búsqueda entrenable que asignará cada ID de carácter a un vector con dimensiones embedding_dim
* tf.keras.layers.GRU: un tipo de RNN con units=rnn_units (podríamos haber usado una capa LSTM).
* tf.keras.layers.Dense: la capa de salida, con *vocab_size* nodos de salida. Produce un logit (antes de que se aplique la función de activación en la capa de salida, la salida se llama logit) por cada carácter del vocabulario. La capa densa nos devolverá la distribución de probabilidades sobre todos sus nodos.

In [37]:
vocab_size = len(vocab) # Longitud del vocabulario
embedding_dim = 256     # Dimensiones de la capa de Embedding
rnn_units = 1024        # Número de unidades RNN

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):                 # Método para inicializar la instancia del modelo
    super().__init__(self)                                                  # Llama al método __init__ de la clase base (tf.keras.Model). Esta línea asegura que se realicen todas las inicializaciones necesarias de la clase base antes de agregar las capas personalizadas.
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)   # La capa de embedding convierte números enteros (identificadores de palabras) en vectores densos de embedding_dim dimensiones. Esto es crucial para representar semánticamente las palabras en un espacio continuo.
    self.gru = tf.keras.layers.GRU(rnn_units,                               # Capa de tipo GRU (Gated Recurrent Unit), una variante de las capas recurrentes en las redes neuronales. La GRU es capaz de capturar patrones secuenciales en los datos
                                   return_sequences=True,                   # Nos devolverá una distribución de probabiliades por cada Paso de Tiempo (Ejemplo 'Hello': T1 H, T2 He, ...)
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs                                                              # Secuencia de entrada al modelo
    x = self.embedding(x, training=training)                                # La secuencia de entrada se pasa a través de la capa de embedding. Esta capa convierte los identificadores de palabras en vectores densos. El parámetro training se utiliza para indicar si el modelo está en modo de entrenamiento o inferencia.
    if states is None:                                                      # Se verifica si se proporciona un estado inicial (states).
      states = self.gru.get_initial_state(x)                                # Si no se proporciona, se obtiene un estado inicial
    x, states = self.gru(x, initial_state=states, training=training)        # La capa GRU procesa la secuencia y devuelve tanto la secuencia resultante (x) como el nuevo estado de la celda (states).
    x = self.dense(x, training=training)                                    # Esta capa produce la salida final del modelo

    if return_state:
      return x, states
    else:
      return x

In [39]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

**Probamos el comportamiento del Modelo:**

In [40]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


Para obtener predicciones reales del modelo, hay que muestrear la distribución de salida para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de caracteres.

Es importante tomar muestras de esta distribución, ya que tomar el argmax de la distribución puede hacer que el modelo se atasque fácilmente en un bucle.

Probamos el primer ejemplo en el lote. Esto nos da, en cada paso de tiempo, una predicción del siguiente índice de caracteres.

In [42]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 7, 31, 40, 39, 53, 49, 23,  5,  7, 55, 16,  0, 32,  2, 49, 14, 45,
       51, 35, 63, 40, 39, 16, 36, 57, 59, 28, 42, 44,  8, 24, 26, 58, 54,
       29,  3, 53, 28, 22, 35, 17, 56, 17, 32, 42, 39, 60, 40, 62, 31, 54,
       43, 37, 63, 10,  5,  6, 61, 36, 31, 12, 10, 57, 52, 24, 60,  0, 10,
       37, 42, 13, 54, 54, 31,  2, 51,  1, 36, 44, 25, 38, 29, 53, 55, 54,
       17, 17,  1, 18, 62, 19, 11, 56, 55, 26, 42, 24, 21, 46, 26])

Podemos decodificarlo para ver el texto predicho por este modelo no entrenado.

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

**Entrenamiento del Modelo**

En este punto, el problema se puede tratar como un problema de clasificación estándar. Dado el estado anterior de RNN y la entrada de este paso de tiempo, prediga la clase del siguiente carácter.

**Añadimos un optimizador y una función de pérdida:**

La función de pérdida estándar tf.keras.losses.sparse_categorical_crossentropy funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que el modelo devuelve logits, hay que configurar el indicador from_logits.

In [44]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [45]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189786, shape=(), dtype=float32)


Un modelo recién inicializado no debería estar demasiado seguro de sí mismo, todos los logits de salida deberían tener magnitudes similares. Para confirmar esto podemos comprobar que la exponencial de la pérdida media es aproximadamente igual al tamaño del vocabulario. Una pérdida mucho mayor significa que el modelo está seguro de sus respuestas incorrectas y está mal inicializado.

In [46]:
tf.exp(example_batch_mean_loss).numpy()

66.00866

Configuramos el procedimiento de entrenamiento utilizando el método tf.keras.Model.compile.

In [47]:
model.compile(optimizer='adam', loss=loss)

**Configuración de Puntos de Control (Checkpoints):**

Usamos un tf.keras.callbacks.ModelCheckpoint para asegurarnos de que los puntos de control se guarden durante el entrenamiento.

In [48]:
checkpoint_dir = './training_checkpoints'                        # Directorio en el que se guardan los Checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Nombre de los ficherso de Checkpoint

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Ejecutamos el Entrenamiento del Modelo:**

In [50]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1831
Epoch 2/20
172/172 [==============================] - 14s 62ms/step - loss: 1.1437
Epoch 3/20
172/172 [==============================] - 12s 62ms/step - loss: 1.1029
Epoch 4/20
172/172 [==============================] - 12s 62ms/step - loss: 1.0595
Epoch 5/20
172/172 [==============================] - 13s 63ms/step - loss: 1.0134
Epoch 6/20
172/172 [==============================] - 13s 62ms/step - loss: 0.9648
Epoch 7/20
172/172 [==============================] - 14s 62ms/step - loss: 0.9139
Epoch 8/20
172/172 [==============================] - 13s 61ms/step - loss: 0.8617
Epoch 9/20
172/172 [==============================] - 12s 62ms/step - loss: 0.8101
Epoch 10/20
172/172 [==============================] - 12s 62ms/step - loss: 0.7600
Epoch 11/20
172/172 [==============================] - 13s 61ms/step - loss: 0.7134
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 0.6711
E

**Generar texto:**

La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle y realizar un seguimiento del estado interno del modelo a medida que se ejecuta.

Cada vez que se llama al modelo, se pasa un texto y un estado interno. El modelo devuelve una predicción para el siguiente carácter y su nuevo estado. Vuelva a pasar la predicción y el estado para continuar generando texto.

El siguiente código hace una predicción de un solo paso.

In [51]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None] # Crea una máscara para prevenir que se generen "[UNK]"
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [52]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Si lo ejecutamos en bucle, podemos generar algo de texto. Mirando el texto generado, vemos que el modelo sabe cuándo usar mayúsculas, hacer párrafos e imita un vocabulario de escritura similar al de Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar oraciones coherentes.

In [53]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
There wanted colours have been so much the house of Lancaster.
Those dwells musicians for authority
As well obey agains; against you
Medward to help! amidity, is not most?
As mistress you mistake.

GLOUCESTER:
My brother for autidity: the whole heart's doors!
Come, follow me, and provide both their faces
Of all your people bend me hut.

AUTOLYCUS:
I am a good Putied and awhile at libe:
Embrace may thither shall have lived to speak fort:
Help, help! Call of the king's,
may song, or wrong too lict but danger
Which he profess'd: for I will not be
That I shall be well beloved.
In the name too well, but lovers'd, thank you
Of what is pasting for us unthere.
Give me thy tongue.

QUEEN MARGARET:
Nay, go not from us thus.
If he were falsehood he rudesty,
To make his valour live it was revolt,
Your children's deeds, to do their abuses;
And that the sky, if Romeo's nose
That torn'd eye or are old, Clubble Edward's moint:
Thou yet to-day upon the bowels of the
see--O, Outhously undrunged m

**Posibilidades de Mejora:**

* Entrenar el modelo durante más épocas.
* Modificar la cadena inicial de datos.
* Experimentar con una capa de inicio diferente. Se puede intente agregar otra capa RNN para mejorar la precisión del modelo o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

Si desea que el modelo genere texto más rápido , lo más fácil que puede hacer es generar el texto por lotes. En el siguiente ejemplo, el modelo genera 5 salidas aproximadamente en el mismo tiempo que se tardó en generar 1 arriba.

In [54]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe devil care from Edward's children keep him in the dire,\nAnd left thee back our highor to the wisest, or our\nMost buried in place. But leave to fly,\nThat Tybalt bend like us, in post,\nSo stands me to the block of state, where nothing\nflames here we weirs our state; feign Albar-m,\nWhere stamed his liberty makes, made place pill's lives\nWith lovers' swords we ento a bower\nThan you shall buzzer, wronging to your charge,\nScalibial hablet read of bitter when which time your bitter\nHe plainers wish the babis.\n\nHASTINGS:\nSo thrive me, death, what was he that utter'd flin?\nReare the valour of my cheeks.\nWhat was my weddod took his life?\nNow prisoners lie thou the rest.\n\nDUKE VINCENTIO:\nMy heart cannot he doubt; for you may know\nHe let us here another affection.\nThis is my orato, and thy face to land a treaty\nAnd do it in the world; and we perching in\nWernabsit, we'll deny thy crown'd\nTo know your father's house: his son is daughter,\nThat when t

**Exportar el Generador de Texto:**

Este modelo de un solo paso se puede guardar y restaurar fácilmente, lo que le permite usarlo en cualquier lugar donde se acepte un tf.saved_model.

In [55]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

Para utilizar el modelo guardado...

In [56]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The contents of it. Didst yet the king
Of all the world to see him: be assured
In any perfect to th


### **Avanzado: Entrenamiento Personalizado** ###

El procedimiento de entrenamiento anterior es simple, pero permite mucho control. Se puede utilizar el maestro forzado que evita que las malas predicciones se retroalimenten al modelo, por lo que el modelo nunca aprende a recuperarse de los errores.

Ahora vamos a implementar un ciclo de entrenamiento. Esto brinda un punto de partida si, por ejemplo, desea implementar el aprendizaje del plan de estudios para ayudar a estabilizar la salida de bucle abierto del modelo.

La parte más importante de un ciclo de entrenamiento personalizado es la función de paso de entrenamiento.

Usamos tf.GradientTape para rastrear los degradados. Para más información sobre este enfoque... [guía de ejecución ansiosa](https://www.tensorflow.org/guide/basics).

El procedimiento básico es:

1. Ejecutar el modelo y calcular la pérdida bajo un tf.GradientTape.
2. Calcular las actualizaciones y aplicarlas al modelo utilizando el optimizador.

In [57]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

La implementación anterior del método train_step sigue las convenciones train_step de Keras. Esto es opcional, pero permite cambiar el comportamiento del paso de tren y seguir usando los métodos Model.compile y Model.fit de keras.

In [58]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [59]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [60]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 62ms/step - loss: 2.7263


Si hace falta más control, se puede escribir un ciclo de entrenamiento personalizado completo.

In [61]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1893
Epoch 1 Batch 50 Loss 2.0712
Epoch 1 Batch 100 Loss 1.9571
Epoch 1 Batch 150 Loss 1.8657

Epoch 1 Loss: 1.9935
Time taken for 1 epoch 13.55 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7771
Epoch 2 Batch 50 Loss 1.7589
Epoch 2 Batch 100 Loss 1.7141
Epoch 2 Batch 150 Loss 1.6366

Epoch 2 Loss: 1.7144
Time taken for 1 epoch 12.10 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5961
Epoch 3 Batch 50 Loss 1.5859
Epoch 3 Batch 100 Loss 1.5854
Epoch 3 Batch 150 Loss 1.4921

Epoch 3 Loss: 1.5525
Time taken for 1 epoch 14.14 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4663
Epoch 4 Batch 50 Loss 1.4946
Epoch 4 Batch 100 Loss 1.4612
Epoch 4 Batch 150 Loss 1.4484

Epoch 4 Loss: 1.4531
Time taken for 1 epoch 13.20 sec
_____________________________________________________________________